# Adding a New Satellite: ICESat2


The process for adding a new satellite can seem quite complicated and involved, so  have detailed here the steps that were required to implement the ICESat2 satellite trajectory analysis into `pygeodyn`.

First, some info to help us better understand the details.

## ICESat2 Details and GEODYN Run Info:

             
             
  - **Date Range of Data**:
      - Data from day 287/2018 until day 019/2019.  
      
      

  - **Sat. Shape and Orbit**:
      - The ICESat2 has a complicated run shape, therefore it requires an external attitude file:
          - `EXAT01.2018.287.gz` --> `EXAT01`  
          
      - Orbital Elements and Satellite details:
          ```
           +————————————————————————————————————————————————————————————————————————————————————+
           | SAT. ID = 1807001  AREA(M**2) =     9.53000  MASS(KG) =    1514.000                |
           |                                                                                    |
           |  THE FOLLOWING ARE GEOCENTRIC LATITUDE AND GEOCENTRIC LONGITUDE AND AN             |
           |  APPROXIMATE HEIGHT FROM THE SEMI-MAJOR AXIS OF THE CENTRAL BODY                   |
           |   LATITUDE =  18.319 DEG   LONGITUDE = 278.129 DEG   HEIGHT =    480945.790 M      |
           |  APOGEE =          476130.962 METERS     ASC NODE RATE =     0.255688 DEG/DAY      |
           |  PERIGEE=          452146.086 METERS     ARG PERG RATE =    -3.831197 DEG/DAY      |
           |  PERIOD =              94.317 MINUTES    PERIOD   RATE =    -0.000000 SEC/DAY      |
           |  DRAG   =               0.000 M/DAY**2   S-M AXIS RATE =    -0.000000 M/DAY        |
           +————————————————————————————————————————————————————————————————————————————————————+
 ```  

      
  - **Tracking Datatype (PCE)**: 
      - the tracking data-type used in a trajectory analysis is called `PCE`.  This is an output from a separate GEODYN run (ran by Code 61) in which reduced-dynamics were applied (i.e. empirical accelerations were turned on) to get an extremely accurate determination of the satellite's orbit.  We can then use this trajecotry/orbit data as a baseline to compare against when running non-reduced dynamics runs with different force models to see how well the force models perform.
          - `g2b_pce.gz` --> `FORT.40`
      - The  time segments for the PCE datatype were written in 30-hour chunks (24 hour segments with 3 hours added to each end to account for end effect).
      
   - **Arcs**
       -54 hour arcs with substantial overlap between them
   
   - **Files**: All data that is used in the ICESat2-PCE GEODYN run
       - ATGRAV:
           - `ATGRAV.glo-3HR_20160101-PRESENT_9999_AOD1B_0006.0090.gz` --> `fort.18`
       - EPHEM
           - `ephem1430.data_2025.gz` --> `ftn01`
       - G2B
           - `icesat2_g2b_PCE_gpstraj.gz` --> `ftn40`
       - GRAVITY
           - `eigen-6c.gfc_20080101_do_200_fix.grv.gz` --> `ftn12`
       - GDNTABLE
           - `gdntable.data` --> `ftn02`
       - Daily dependence:
            - setup:
                - `iisset.YYYY.DDD`  --> `ftn05`
            - EXTERNAL ATTITUDE:
               - `EXAT01.YYYY.DDD.gz` --> `EXAT01`



##  Steps taken to include the ICESat2 Satellite:

1. Preprocessing
   1. Place all data in appropriate locations
   2. Convert "raw" trajectory data into readable PCE format
   3. Construct a `G2B` PCE file and put it in the correct location. 
        - this will contain all days  
       
2.  Construct an ICESat2 satellite class 
    1. The initialization of this class will contain all the ICESat2 satellite details and file info:
        ```
        def __init__(self,params):
            super().__init__(params)

            #### HARD CODE the satellite properties
            self.SATELLITE_dir = 'icesat2'
            self.SATID         = '1807001'
            self.YR            = 2018
            self.DATA_TYPE     = 'PCE'
            self.grav_id = '' 

            self.g2b_file = 'icesat2_g2b_PCE_gpstraj.gz'
            self.atgrav_file = 'ATGRAV.glo-3HR_20160101-PRESENT_9999_AOD1B_0006.0090.gz'
            self.ephem_file = 'ephem1430.data_2025.gz'
            self.gravfield_file = 'eigen-6c.gfc_20080101_do_200_fix.grv.gz'
            self.external_attitude = 'EXAT01.2018.287.gz'

            ### Modifications that are ICESat2 Specific:
            self.ARC =   self.arc_input
        ```  
        
    2. Overwrite CONTROLLER methods: The following methods (class functions) are overwritten within this class to account for ICEsat2 specific needs:  
        
        - `clean_iisset_file()`
            - the iisset setup file is modified to work for the type of run we want to do for this satellite
        - `print_runparameters_to_notebook()`
            - the original method is overwritten to account for the fact that we have a external attitude file
        - `prepare_tmpdir_for_geodyn_run()`
            - this method is overwritten to include the external attitude file

    3. Overwrite READER methods: The following methods (class functions) are overwritten within this class to account for ICEsat2 specific needs:  

        - `set_file_paths_for_multiple_arcs()`
            - this method is overwritten to account for the different naming conventions for the icesat2 arcs
    4. Minor modification to the BASE FUNCTIONS in the reader.
        - `read_observed_resids()`
            - Added the PCE datatype when reading in the headers from the IIEOUT file.  This allows us to save out the residual data under the different kinds of header types.
        - improved the overall method for reading in the final iteration number to account for double digits.
    

##  The Setup file:

If you get an error check the following:
 - there have to be 3 lines at the top of the file
 - check the dates on the modified cards that require dates
